<a href="https://colab.research.google.com/github/audrb1999/Python_image/blob/main/git_hub12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***14. 중급 Captcha Hacking ④ - KNN 모델 학습*** 
파이참에서 이용해야함!

In [ ]:
import os
import cv2
import numpy as np

file_names = list(range(0, 13))
train = []
train_labels = []# 데이터 라벨 설정

for file_name in file_names:
    path = 'training_data/' + str(file_name) + '/'
    file_count = len(os.listdir('training_data/' + str(file_name) + '/'))# 총 이미지 갯수를 읽어롬
    for i in range(1, file_count + 1):# 그 이미지 갯수만큼 반복
        img = cv2.imread(path + str(i) + '.png')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)# 읽은 파일을 흑백으로 바꿈
        train.append(gray)#
        train_labels.append(file_name)


x = np.array(train)
train = x[:, :].reshape(-1, 400).astype(np.float32)# 20x20 을 1차원으로 변경
train_labels = np.array(train_labels)[:, np.newaxis]# 2차원으로 변형

print(train.shape)# 우리가 수집한 데이터, 24개에 길이 400
print(train_labels.shape)

np.savez('train.npz', train = train, train_labels = train_labels)#데이터 셋 저장!



KNN 알고리즘은 머신러닝 알고리즘이지만 이 정도만해도 학습이 끝남!

가볍게 실행 시켜보자!

In [ ]:
import numpy as np
import cv2
import utils

FILE_NAME = 'train.npz'# 데이터 셋 가져와서

#각 글자의 (1 x 400) 데이터와 정답 (0~9, +-*)
with np.load(FILE_NAME) as data:# 그 데이터 셋을 data라는 변수에 넣고
    train = data['train']# 옮겨 담음
    train_labels = data['train_labels']

knn = cv2.ml.KNearest_create()#knn 선언
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)#knn 학습

def check(test, train, train_labes):
    # test 이미지와 가장 가까운 레이블을 출력 k = 1인 이유는 모든 이미지가 동일한 크기, 동일한 모양이라서 가장 가까운거 하나 찾아도 됨!
    #가장 가까운 k개의 글자를 찾아, 어떤 숫자에 해당하는지 찾습니다. 테스트 데이터 갯수에 따라 조절
    ret, result, neighbours, dist = knn.findNearest(test, k=1)
    return result

def get_result(file_name):# 파일이 들어오면
    image = cv2.imread(file_name)
    chars = utils.extract_chars(image)# 각각의 이미지를 왼쪽부터 담고
    result_string = ""
    for char in chars:
        # 이미지를 400으로 쭉 늘어트리고 매치되는 데이터를 담음
        matched = check(utils.resize20(char[1]), train, train_labels)
        if matched < 10:
            # 숫자를 왼쪽부터 차례로 붙일 수 있게
            result_string += str(int(matched))
            continue
            
        if matched == 10:
            matched = "+"

        elif matched == 11:
            matched = "+"

        elif matched == 12:
            matched = "+"

        result_string += matched
    return result_string
# 최종적으로 이미지를 문자열로 반환
print(get_result("3.png"))
